# Start - Zellen werden aktiviert mit Shift + Enter

In [1]:
from openpyxl import load_workbook
from pathlib import Path
from datetime import date
import calendar
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.table import WD_TABLE_ALIGNMENT
from win32com.client import Dispatch
import shutil
import uuid
import time

# ----------------- KONFIGURATION -----------------

1. report_monat auf den richtigen monat einstellen
2. Überprüfen ob Excel pfad richtig ist
3. template_path überprüfen
4. out_dir richtig setzen

In [2]:

report_monat = "2020-03"  # YYYY-MM
year, month = map(int, report_monat.split("-"))
mon_str = date(year, month, 1).strftime("%B %Y")

EXCEL_PATH   = Path(r'XXXXXXXXX\OneDrive - AGGM\AGGM\2025\00_NK/Netzverlustenergie 2024_test.xlsx')
TEMPLATE_PATH = Path(r"XXXXXXXXX\OneDrive - AGGM\AGGM\2025\04_Berichte/NEB-Bericht.docx")
OUT_DIR      = Path(str(Path(r"XXXXXXXXX\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte")) +'/'  + report_monat)
OUT_DIR.mkdir(parents=True, exist_ok=True)





# ----------------- COM-Funktion mit Aktivierung -----------------

In [3]:
def convert_to_pdf(docx_path: Path, pdf_path: Path):
    """
    Exportiert ein DOCX zu PDF via Word-COM über SaveAs.
    Umgeht COM-RPC-Fehler durch direkte SaveAs.
    """
    tmp_docx = docx_path.parent / f"tmp_{uuid.uuid4().hex}.docx"
    shutil.copy(docx_path, tmp_docx)
    word = Dispatch('Word.Application')
    word.Visible = False
    doc = word.Documents.Open(str(tmp_docx), ReadOnly=False)
    time.sleep(0.5)
    doc.SaveAs(str(pdf_path), FileFormat=17)  # wdFormatPDF
    doc.Close(False)
    word.Quit()
    tmp_docx.unlink()

# ----------------- Platzhalter ersetzen -----------------
def replace_placeholders(doc: Document, context: dict):
    for p in doc.paragraphs:
        for key, val in context.items():
            placeholder = f"{{{{{key}}}}}"
            if placeholder in p.text:
                p.text = p.text.replace(placeholder, str(val))
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for key, val in context.items():
                    placeholder = f"{{{{{key}}}}}"
                    if placeholder in cell.text:
                        cell.text = cell.text.replace(placeholder, str(val))

# ----------------- Literal 'a' ersetzen -----------------
def replace_literal_a(doc: Document, name: str):
    for p in doc.paragraphs:
        if p.text.strip() == 'a':
            p.text = name
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if cell.text.strip() == 'a':
                    cell.text = name

# ----------------- Tabellen-Formatierung -----------------
def format_kwh(val):
    num = float(val or 0)
    s = f"{num:,.3f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    return f"{s} kWh"

def format_pct(val, is_summary=False):
    num = float(val or 0) * 100
    s = f"{num:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    return (f"Ø {s}%" if is_summary else f"{s}%")

def style_and_fill(table, rows, summary):
    table.style = 'Table Grid'
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    table.autofit = False
    widths = [Inches(1.2)] * 4
    for idx, width in enumerate(widths):
        for cell in table.columns[idx].cells:
            cell.width = width
    # clear existing data rows
    while len(table.rows) > 1:
        table._tbl.remove(table.rows[-1]._tr)
    # fill rows
    for r in rows:
        cells = table.add_row().cells
        cells[0].text = r['Datum'].strftime('%d.%m.%Y')
        cells[1].text = format_kwh(r['Prognose'])
        cells[2].text = format_kwh(r['Clearing'])
        cells[3].text = format_pct(r['Fehler'])
        for cell in cells:
            for run in cell.paragraphs[0].runs:
                run.font.size = Pt(8)
    # summary
    sumcells = table.add_row().cells
    sumcells[0].text = 'Gesamt'
    sumcells[1].text = format_kwh(summary['Prognose'])
    sumcells[2].text = format_kwh(summary['Clearing'])
    sumcells[3].text = format_pct(summary['Fehler'], is_summary=True)
    for cell in sumcells:
        for run in cell.paragraphs[0].runs:
            run.font.size = Pt(8)
            run.font.highlight_color = 7

# ----------------- Excel-Daten auslesen --------------------

In [4]:
wb = load_workbook(EXCEL_PATH, data_only=True)
ws = wb['Prognosegüte (d)']  # Name anpassen

# finde Operatoren mit JA
ops = []
for c in range(3, ws.max_column + 1, 3):
    if ws.cell(row=1, column=c + 2).value == 'JA':
        ops.append({'Name': ws.cell(row=1, column=c).value, 'col': c})

# finde Header-Zeile (erste 'Prognose')
header = next(r for r in range(1, ws.max_row + 1)
              if ws.cell(row=r, column=ops[0]['col']).value == 'Prognose')

# finde Start-Zeile für das erste Datum des Monats (vergleich als date)
from datetime import datetime as _dt

first_date = date(year, month, 1)
row_start = None
for r in range(header + 1, ws.max_row + 1):
    val = ws.cell(row=r, column=2).value
    # konvertiere datetime zu date für den Vergleich
    if hasattr(val, 'date'):
        cell_date = val.date()
    else:
        cell_date = val
    if cell_date == first_date:
        row_start = r
        break
if row_start is None:
    raise ValueError(f"Startdatum {first_date} nicht gefunden in Spalte B")

days = calendar.monthrange(year, month)[1]  # Anzahl Tage im Berichtmonat

# Gesamt-Daten einlesen
# Finde Gesamt-Spalte
tot_col = next((c for c in range(1, ws.max_column + 1)
                if isinstance(ws.cell(row=1, column=c).value, str)
                and 'GESAMT AGGN_NV' in ws.cell(row=1, column=c).value.strip().upper()),
               ws.max_column - 2)

tot_rows = []
for i in range(days):
    r = row_start + i
    tot_rows.append({
        'Datum': date(year, month, i + 1),
        'Prognose': ws.cell(row=r, column=tot_col).value,
        'Clearing': ws.cell(row=r, column=tot_col + 1).value,
        'Fehler': ws.cell(row=r, column=tot_col + 2).value
    })

# Summen
tot_sum = {
    'Prognose': sum(r['Prognose'] or 0 for r in tot_rows),
    'Clearing': sum(r['Clearing'] or 0 for r in tot_rows),
    'Fehler': sum(r['Fehler'] or 0 for r in tot_rows) / days
}

# Berichte erstellen
for op in ops:
    rows = []
    for i in range(days):
        r = row_start + i
        rows.append({
            'Datum': date(year, month, i + 1),
            'Prognose': ws.cell(row=r, column=op['col']).value,
            'Clearing': ws.cell(row=r, column=op['col'] + 1).value,
            'Fehler': ws.cell(row=r, column=op['col'] + 2).value
        })
    summ = {
        'Prognose': sum(r['Prognose'] or 0 for r in rows),
        'Clearing': sum(r['Clearing'] or 0 for r in rows),
        'Fehler': sum(r['Fehler'] or 0 for r in rows) / days
    }

    doc = Document(TEMPLATE_PATH)
    replace_placeholders(doc, {'Monat': mon_str, 'VNB': op['Name']})
    replace_literal_a(doc, op['Name'])
    style_and_fill(doc.tables[0], rows, summ)
    style_and_fill(doc.tables[1], tot_rows, tot_sum)

    filename = f"{year:04d}-{month:02d} NEB Bericht {op['Name']}"
    p_docx = OUT_DIR / f"{filename}.docx"
    p_pdf = OUT_DIR / f"{filename}.pdf"
    doc.save(p_docx)
    convert_to_pdf(p_docx, p_pdf)
    print(f"fertig: {p_pdf}")


fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht Elektrizitätswerk Wels AG.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht Energie Graz GmbHCo KG.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht Energie Klagenfurt GmbH.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht Energienetze Steiermark GmbH.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht Gasnetz Veitsch.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht KNGKrnten Netz GmbH.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\2020-03 NEB Bericht LINZ GAS Netz GmbH.pdf
fertig: C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Beri

In [40]:
import os
import glob
import re
from docx import Document
from pathlib import Path

# 1) Mapping von Kapitel-Überschrift → Suchschlüssel im Dateinamen
FILENAME_MAP = {
    "eww AG":                         "Elektrizitätswerk Wels AG",
    "Energie Graz GmbH & Co KG":     "Energie Graz GmbHCo KG",
    "KNG-Kärnten Netz GmbH":         "KNGKrnten Netz GmbH",
    "Netz Niederösterreich GmbH":    "Netz Niedersterreich GmbH",
    "Netz Oberösterreich GmbH":      "O Ferngas Netz GmbH",
    # bei Bedarf weitere Einträge ergänzen...
}

def merge_tables_per_operator(
    overview_template: str,
    reports_dir: str,
    output_file: str
):
    """
    Lädt das Übersichtstemplate (NEB-Bericht-EC.docx),
    iteriert alle Level-2-Headings (5.1, 5.2, …),
    und fügt darunter die Tabellen aus den Einzelberichten
    ein (immer alle außer der letzten Zusammenfassungstabelle).
    """
    overview = Document(overview_template)
    body     = overview._element.body

    for para in overview.paragraphs:
        style = para.style.name
        if style in ("Heading 2", "Überschrift 2"):
            op_name = para.text.strip()
            if not op_name:
                continue

            # 1) Mapping anwenden (falls definiert), sonst Kapitel-Text verwenden
            key = FILENAME_MAP.get(op_name, op_name)

            # 2) Nach passender .docx suchen
            pattern = f"*NEB Bericht*{key}*.docx"
            matches = glob.glob(os.path.join(reports_dir, pattern))
            if not matches:
                print(f"⚠️ Kein Report gefunden für „{op_name}“ (Muster: {pattern})")
                continue

            report_path = matches[0]
            rpt         = Document(report_path)

            # 3) Einfüge-Index direkt nach diesem Heading
            insert_idx = list(body).index(para._p) + 1

            # 4) Alle Tabellen bis auf die letzte einfügen
            for tbl in rpt.tables[:-1]:
                body.insert(insert_idx, tbl._tbl)
                insert_idx += 1

    # 5) Speichern
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    overview.save(output_file)
    print(f"✅ Übersicht mit allen Tabellen unter „{output_file}“ erstellt.")

if __name__ == '__main__':
    report_monat = report_monat
    TEMPLATE     = r"C:XXXXXXX\OneDrive - AGGM\AGGM\2025\04_Berichte\NEB-Bericht-EC_2.docx"
    REPORTS_DIR  = Path(r"XXXXXXX\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte") / report_monat

    # Ausgabe-Verzeichnis & Datei
    out_dir = REPORTS_DIR
    out_dir.mkdir(parents=True, exist_ok=True)
    file_name = "Bericht_EC" + "_"+ report_monat + ".docx"
    OUTPUT = out_dir / file_name

    merge_tables_per_operator(str(TEMPLATE), str(REPORTS_DIR), str(OUTPUT))

⚠️ Kein Report gefunden für „Portfolioeffekt“ (Muster: *NEB Bericht*Portfolioeffekt*.docx)
⚠️ Kein Report gefunden für „Evaluierung der Beschaffung“ (Muster: *NEB Bericht*Evaluierung der Beschaffung*.docx)
⚠️ Kein Report gefunden für „Prognosequalität“ (Muster: *NEB Bericht*Prognosequalität*.docx)
✅ Übersicht mit allen Tabellen unter „C:\Users\johannes.misensky\OneDrive - AGGM\AGGM\2025\00_NK\4 Berichte\2020-03\Bericht_EC_2020-03.docx“ erstellt.


In [39]:
"Bericht_EC" + "_"+ report_monat + ".docx"

'Bericht_EC_2020-03.docx'